# TODO
- Use "speed" vector to restrict ripple detection to NREM

In [78]:
%matplotlib widget
import ipywidgets as widgets

In [79]:
from ecephys.sglx_utils import makeMemMapRaw, readMeta, SampRate, GainCorrectIM
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [80]:
from ripple_detection.core import ripple_bandpass_filter
from scipy.signal import filtfilt

In [81]:
import pandas as pd

## Load the data

In [82]:
binPath = Path('/Volumes/neuropixel/Data/CNPIX6-Eugene/9.24.2020_SR_24hs_g0_t0.imec0.lf.bin')
tStart = None
tEnd = None
# chanList = np.asarray([161, 162, 165, 166, 169, 170, 173, 174, 177, 178, 181]) # Doppio
# chanList = np.asarray([162, 165, 166, 169, 170, 173, 174, 177, 178, 181, 182]) # Valentino
# chanList = np.asarray([159, 161, 163, 165, 167, 169, 171, 173, 175, 177, 179]) # Alessandro
# chanList = np.asarray([163, 165, 167, 169, 171, 173, 175, 177, 179, 181, 183]) #Segundo
chanList = np.asarray([193, 195, 197, 199, 201, 203, 205, 207, 209, 211]) #Eugene

In [83]:
def load_data(binPath, chanList, tStart=None, tEnd=None):
    meta = readMeta(binPath)
    rawData = makeMemMapRaw(binPath, meta)
    sRate = SampRate(meta)
    
    # Calculate desire start and end samples
    if tStart:
        firstSamp = int(sRate*tStart)
    else:    
        firstSamp = 0
        
    if tEnd:
        lastSamp = int(sRate*tEnd)
    else:
        nFileChan = int(meta['nSavedChans'])
        nFileSamp = int(int(meta['fileSizeBytes'])/(2*nFileChan))
        lastSamp = nFileSamp - 1
    
    # array of times for plot
    tDat = np.arange(firstSamp, lastSamp+1)
    tDat = 1000*tDat/sRate      # plot time axis in msec

    selectData = rawData[chanList, firstSamp:lastSamp+1]
    if meta['typeThis'] == 'imec':
        # apply gain correction and convert to uV
        convData = 1e6*GainCorrectIM(selectData, chanList, meta)
    else:
        MN, MA, XA, DW = ChannelCountsNI(meta)
        # print("NI channel counts: %d, %d, %d, %d" % (MN, MA, XA, DW))
        # apply gain coorection and conver to mV
        convData = 1e3*GainCorrectNI(selectData, chanList, meta)
        
    return tDat, convData.T, sRate

In [84]:
(time, lfps, sampling_frequency) = load_data(binPath, chanList, tStart, tEnd)

nChan: 385, nFileSamp: 9000026


In [47]:
lfps_df = pd.DataFrame(lfps, columns=chanList, index=pd.Index(time, name='time'))
lfps_df.plot(subplots=True, figsize=(18, 6))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Load hypnogram and select only NREM epochs

In [86]:
hypnoPath = Path('/Volumes/neuropixel/Data/CNPIX6-Eugene/9.24.2020_SR_24hs_g0_t0.hypnogram.txt')
hypno = pd.read_csv(hypnoPath, sep='\t', names=['state', 'end_time'], skiprows=1)
hypno['start_time'] = hypno.apply(lambda x: 0 if x.name == 0 else hypno.loc[x.name - 1].end_time, axis=1)
hypno

,state,end_time,start_time
0,N1,7.990000,0.000000
1,N2,23.990000,7.990000
2,N1,39.990002,23.990000
3,N2,243.990005,39.990002
4,N1,251.990005,243.990005
...,...,...,...
88,N1,3567.989990,3543.989990
89,Wake,3575.989990,3567.989990
90,N1,3583.989990,3575.989990
91,N2,3591.989990,3583.989990


In [87]:
speed = np.full_like(time, np.inf)
NREM = hypno.loc[hypno['state'] == 'N2']
for epoch in NREM.itertuples():
    speed[np.where(np.logical_and(time >= epoch.start_time * 1000, time <= epoch.end_time * 1000))] = 0

## Apply ripple bandpass filter

In [88]:
def apply_ripple_filter(lfps, sampling_frequency):
    filter_numerator, filter_denominator = ripple_bandpass_filter(sampling_frequency)
    is_nan = np.any(np.isnan(lfps), axis=-1)
    filtered_lfps = np.full_like(lfps, np.nan)
    filtered_lfps[~is_nan] = filtfilt(filter_numerator, filter_denominator, lfps[~is_nan], axis=0)
    return filtered_lfps

In [89]:
filtered_lfps = apply_ripple_filter(lfps, sampling_frequency)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 6))
ax1.plot(time, lfps)
ax2.plot(time, filtered_lfps)
plt.show()

## Detect Ripples

In [ ]:
def plot_ripples(time, lfps, filtered_lfps, ripple_times):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 6))
    
    ax1.plot(time, filtered_lfps)
    for ripple in ripple_times.itertuples():
        ax1.axvspan(ripple.start_time, ripple.end_time, alpha=0.3, color='red', zorder=1000)

    ax2.plot(time, lfps)
    for ripple in ripple_times.itertuples():
        ax2.axvspan(ripple.start_time, ripple.end_time, alpha=0.3, color='red', zorder=1000)
        
    plt.show()

### Kay method
Detection of SWRs was prerequisite for all data analyzed in this study, and was performed only when at least three CA1 cell layer recordings were available. Offline, a multisite average approach was used to detect SWRs57. Specifically, LFPs from all available CA1 cell layer tetrodes were filtered between 150–250 Hz, then squared and summed across tetrodes. This sum was smoothed with a Gaussian kernel (σ = 4 ms) and the square root of the smoothed sum was analyzed. SWRs were detected when the signal exceeded 2 s.d. of the recording epoch mean for at least 15 ms. SWR periods were then defined as the periods, containing the times of threshold crossing, in which the power trace exceeded the mean. SWR onset was defined as the start of a SWR period. Detection of SWRs was performed only when subjects' head speed was <4 cm/s. For SWR-triggered spike raster plots and PSTH plots, a 0.5 s exclusion period was imposed to isolate SWRs occurring only after non-SWR periods; otherwise, analyses of SWRs included all detected SWRs.

In [90]:
from ripple_detection import Kay_ripple_detector

Kay_ripple_times = Kay_ripple_detector(time, filtered_lfps, speed, sampling_frequency, zscore_threshold=3)
display(Kay_ripple_times)

,start_time,end_time
ripple_number,,
1,9.706373e+03,9.840773e+03
2,1.036317e+04,1.045557e+04
3,1.046077e+04,1.052157e+04
4,1.480756e+04,1.493996e+04
5,1.672795e+04,1.677875e+04
...,...,...
1335,3.347662e+06,3.347700e+06
1336,3.584058e+06,3.584155e+06
1337,3.586845e+06,3.586947e+06


In [ ]:
plot_ripples(time, lfps, filtered_lfps, Kay_ripple_times)

### Karlsson method
Sharp waves ripple events (SWRs) were identified based on peaks in the local field potential (LFP) recorded from one channel from each tetrode in the CA3 and CA1 cell layers. The raw LFP data were band-pass filtered between 150–250 Hz and the SWR envelope was determined using a Hilbert transform. The envelope was smoothed with a Gaussian (4 ms stdev). We initially identified SWR events as sets of times when the smoothed envelope stayed above 3 standard deviations of the mean for at least 15 ms on at least one tetrode. We defined the entire SWR as including times immediately before and after that threshold crossing event during which the envelope exceeded the mean. Overlapping SWRs were combined across tetrodes, so many events extended beyond the SWR seen on a single tetrode.

In [ ]:
from ripple_detection import Karlsson_ripple_detector

Karlsson_ripple_times = Karlsson_ripple_detector(time, filtered_lfps, speed, sampling_frequency)
display(Karlsson_ripple_times)

In [ ]:
plot_ripples(time, lfps, filtered_lfps, Karlsson_ripple_times)

### Eschenko-like method
SPW-Rs were detected by means of an automatic thresholding algorithm (Siapas and Wilson 1998; Csicsvari et al. 1999), illustrated in Figure 1. The hippocampal LFP signal was first filtered (150–250 Hz) and the root mean square (RMS) was calculated at every 5 msec (in a moving 10-msec window). The threshold for ripple peak detection was set to 4 SDs above the mean RMS signal. The beginning and end of a ripple were marked at points at which the RMS signal dropped below 2 SD provided that these two points were separated by 25 msec; these time points were used to estimate ripple duration. For every marked ripple, the troughs of the ripples were detected as the minima of the filtered LFP signal, and the deepest trough was marked as the time point representing the respective SPW-R event. The magnitude of the ripple was determined by integrating the RMS signal over the duration of each ripple (King et al. 1999). Ripples were detected during SWS periods only. The ripple amplitude was estimated as a voltage difference between maximum and minimum peaks of the filtered CA1 signal during the ripple event. Ripple density was defined as the number of ripples/second. Figure 1A shows the raw data from a hippocampal electrode. Ripple parameters and how they were measured are illustrated in Figure 1B.

In [103]:
from ripple_detection.core import (segment_boolean_series, _extend_segment)

def extend_detection_threshold_to_boundaries(is_above_boundary_threshold, is_above_detection_threshold, time,
                                             minimum_duration=0.015):
    '''Extract segments above threshold if they remain above the threshold
    for a minimum amount of time and extend them to a boundary threshold.
    
    Parameters
    ----------
    is_above_boundary_threshold : ndarray, shape (n_time,)
        Time series indicator function specifying when the
        time series is above the boundary threshold.
    is_above_detection_threshold : ndarray, shape (n_time,)
        Time series indicator function specifying when the
        time series is above the the detection_threshold.
    time : ndarray, shape (n_time,)
    
    Returns
    -------
    candidate_ripple_times : list of 2-element tuples
        Each tuple is the start and end time of the candidate ripple.
    '''
    is_above_detection_threshold = pd.Series(is_above_detection_threshold, index=time)
    is_above_boundary_threshold = pd.Series(is_above_boundary_threshold, index=time)
    above_boundary_segments = segment_boolean_series(
        is_above_boundary_threshold, minimum_duration=minimum_duration)
    above_detection_segments = segment_boolean_series(
        is_above_detection_threshold, minimum_duration=minimum_duration)
    return sorted(
        _extend_segment(above_detection_segments, above_boundary_segments))

In [104]:
from scipy.stats import zscore

def _threshold_by_zscore(data, time, minimum_duration=0.015,
                         detection_zscore_threshold=2, boundary_zscore_threshold=0):
    '''Standardize the data and determine whether it is above a given
    number.
    
    Parameters
    ----------
    data : array_like, shape (n_time,)
    detection_zscore_threshold : int, optional
    boundary_zscore_threshold: int, optional
    
    Returns
    -------
    candidate_ripple_times : pandas Dataframe
    '''
    zscored_data = zscore(data)
    is_above_boundary_threshold = zscored_data >= boundary_zscore_threshold
    is_above_detection_threshold = zscored_data >= detection_zscore_threshold

    return extend_detection_threshold_to_boundaries(
        is_above_boundary_threshold, is_above_detection_threshold, time,
        minimum_duration=minimum_duration)

In [105]:
from ripple_detection.core import (exclude_close_events, exclude_movement, gaussian_smooth,
                                   get_envelope)

def generalized_Kay_ripple_detector(time, filtered_lfps, speed, sampling_frequency,
                                    speed_threshold=4.0, minimum_duration=0.015,
                                    detection_zscore_threshold=2.0, boundary_zscore_threshold=0.0,
                                    smoothing_sigma=0.004, close_ripple_threshold=0.0):
    '''Find start and end times of sharp wave ripple events (150-250 Hz)
    based on the Kay method, but allowing boundaries to be defined by Z-scores
    rather than the mean. 
    
    Parameters
    ----------
    time : array_like, shape (n_time,)
    filtered_lfps : array_like, shape (n_time, n_signals)
        Bandpass filtered time series of electric potentials in the ripple band
    speed : array_like, shape (n_time,)
        Running speed of animal
    sampling_frequency : float
        Number of samples per second.
    speed_threshold : float, optional
        Maximum running speed of animal for a ripple
    minimum_duration : float, optional
        Minimum time the z-score has to stay above threshold to be
        considered a ripple. The default is given assuming time is in
        units of seconds.
    detection_zscore_threshold : float, optional
        Number of standard deviations the ripple power must exceed to
        be considered a ripple
   boundary_zscore_threshold : float, optional
        Number of standard deviations the ripple power must drop 
        below to define the ripple start or end time. 
    smoothing_sigma : float, optional
        Amount to smooth the time series over time. The default is
        given assuming time is in units of seconds.
    close_ripple_threshold : float, optional
        Exclude ripples that occur within `close_ripple_threshold` of a
        previously detected ripple.
        
    Returns
    -------
    ripple_times : pandas DataFrame
    '''
    filtered_lfps = np.asarray(filtered_lfps)
    not_null = np.all(pd.notnull(filtered_lfps), axis=1) & pd.notnull(speed)
    filtered_lfps, speed, time = (
        filtered_lfps[not_null], speed[not_null], time[not_null])

    filtered_lfps = get_envelope(filtered_lfps)
    combined_filtered_lfps = np.sum(filtered_lfps ** 2, axis=1)
    combined_filtered_lfps = gaussian_smooth(
        combined_filtered_lfps, smoothing_sigma, sampling_frequency)
    combined_filtered_lfps = np.sqrt(combined_filtered_lfps)
    
    candidate_ripple_times = _threshold_by_zscore(
        combined_filtered_lfps, time, minimum_duration, 
        detection_zscore_threshold, boundary_zscore_threshold)
    
    ripple_times = exclude_movement(
        candidate_ripple_times, speed, time,
        speed_threshold=speed_threshold)
    ripple_times = exclude_close_events(
        ripple_times, close_ripple_threshold)
    index = pd.Index(np.arange(len(ripple_times)) + 1,
                     name='ripple_number')
    return pd.DataFrame(ripple_times, columns=['start_time', 'end_time'],
                        index=index)

In [112]:
Eschenko_ripple_times = generalized_Kay_ripple_detector(time, filtered_lfps, speed, sampling_frequency, detection_zscore_threshold=4.0, boundary_zscore_threshold=1.0, minimum_duration=0.025)
display(Eschenko_ripple_times)

,start_time,end_time
ripple_number,,
1,9.753173e+03,9.808373e+03
2,1.037237e+04,1.043917e+04
3,1.046717e+04,1.049477e+04
4,1.486356e+04,1.493596e+04
5,1.994234e+04,1.997234e+04
...,...,...
970,3.345980e+06,3.346048e+06
971,3.584062e+06,3.584114e+06
972,3.586849e+06,3.586938e+06


In [ ]:
plot_ripples(time, lfps, filtered_lfps, Eschenko_ripple_times)

## Collect ripple statistics

In [91]:
from ripple_detection.core import (gaussian_smooth, get_envelope)

In [92]:
def get_ripple_rms_stats(time, filtered_lfps, ripple_times):
    '''Get statistics about the root-mean-square of LFPs 
    during ripple events.
    
    Parameters
    ----------
    time : array_like, shape (n_time,)
    filtered_lfps : array_like, shape (n_time, n_signals)
        Bandpass filtered time series of electric potentials in the ripple band
    sampling_frequency : float
        Number of samples per second.
    ripple_times: DataFrame, shape (n_ripples, )
        Ripple start and end times, as returned by a detector. 
        
    Returns
    -------
    ripple_stats : pandas DataFrame
        Contains the mean (across channels) RMS of the LFP during each ripple event.
        Contains the summed (across channels) RMS of the LFP during each ripple event. 
        Contains the max (across channels) RMS of the LFP during each ripple event. 
    '''
    ripple_stats = pd.DataFrame(columns = ['mean_rms', 'summed_rms', 'max_rms'], index = ripple_times.index) 
    filtered_lfps = pd.DataFrame(filtered_lfps, index=pd.Index(time, name='time'))

    for ripple in ripple_times.itertuples():
        ripple_lfps = filtered_lfps[ripple.start_time: ripple.end_time]
        ripple_lfps_rms = np.sqrt(np.mean(ripple_lfps**2))
        ripple_stats.loc[ripple.Index] = [np.mean(ripple_lfps_rms), np.sum(ripple_lfps_rms), np.max(ripple_lfps_rms)]
        
    return ripple_stats

In [93]:
def get_Kay_ripple_stats(time, filtered_lfps, sampling_frequency, ripple_times, smoothing_sigma=0.004):
    '''Get statistics for ripples detected using the Kay method, 
    where several channels are summed and their combined envelope 
    is used for ripple detection. All parameters should be as passed
    to the Kay ripple detector. 
    
    Parameters
    ----------
    time : array_like, shape (n_time,)
    filtered_lfps : array_like, shape (n_time, n_signals)
        Bandpass filtered time series of electric potentials in the ripple band
    sampling_frequency : float
        Number of samples per second.
    ripple_times: DataFrame, shape (n_ripples, )
        Ripple start and end times, as returned by the Kay detector. 
    smoothing_sigma : float, optional
        Amount to smooth the time series over time. The default is
        given assuming time is in units of seconds.
        
    Returns
    -------
    ripple_stats : pandas DataFrame
        Contains the integral of the envelope of the combined, filtered lfp.
        Also contains the size of this envelope's peak. 
    '''
    ripple_stats = pd.DataFrame(columns = ['envelope_integral', 'envelope_peak'], index = ripple_times.index) 
    
    filtered_lfps = get_envelope(filtered_lfps)
    combined_filtered_lfps = np.sum(filtered_lfps ** 2, axis=1)
    combined_filtered_lfps = gaussian_smooth(combined_filtered_lfps, smoothing_sigma, sampling_frequency)
    combined_filtered_lfps = np.sqrt(combined_filtered_lfps)
    
    combined_filtered_lfps = pd.DataFrame(combined_filtered_lfps, index=pd.Index(time, name='time'))
    for ripple in ripple_times.itertuples():
        combined_ripple_envelope = combined_filtered_lfps[ripple.start_time: ripple.end_time]
        ripple_stats.loc[ripple.Index] = [np.sum(combined_ripple_envelope[0]), np.max(combined_ripple_envelope[0])]
        
    return ripple_stats

In [94]:
def get_Karlsson_ripple_stats(time, filtered_lfps, sampling_frequency, ripple_times, smoothing_sigma=0.004):
    '''Get statistics for ripples detected using the Karlsson method, 
    where envelopes are calculated separate for each channel and then
    used for ripple detection. All parameters should be as passed
    to the Karlsson ripple detector. 
    
    Parameters
    ----------
    time : array_like, shape (n_time,)
    filtered_lfps : array_like, shape (n_time, n_signals)
        Bandpass filtered time series of electric potentials in the ripple band
    sampling_frequency : float
        Number of samples per second.
    ripple_times: DataFrame, shape (n_ripples, )
        Ripple start and end times, as returned by the Karlsson detector. 
    smoothing_sigma : float, optional
        Amount to smooth the time series over time. The default is
        given assuming time is in units of seconds.
        
    Returns
    -------
    ripple_stats : pandas DataFrame
        Contains the mean (across channels) integral of the envelope of the filtered lfp. 
        Contains the summed (across channels) integrals of the envelope of the filtered lfp. 
        Contains the max (across channels) peak of the envelope of the filtered lfp. 
    '''
    ripple_stats = pd.DataFrame(columns = ['mean_envelope_integral', 'summed_envelope_integrals', 'max_envelope_peak'], index = ripple_times.index) 
    
    filtered_lfps = get_envelope(filtered_lfps)
    filtered_lfps = gaussian_smooth(
        filtered_lfps, sigma=smoothing_sigma,
        sampling_frequency=sampling_frequency)
    
    filtered_lfps = pd.DataFrame(filtered_lfps, index=pd.Index(time, name='time'))
    for ripple in ripple_times.itertuples():
        ripple_envelopes = np.asarray(filtered_lfps[ripple.start_time: ripple.end_time])
        ripple_stats.loc[ripple.Index] = [np.mean(np.sum(ripple_envelopes, axis=0)), np.sum(ripple_envelopes), np.max(ripple_envelopes)]
        
    return ripple_stats

In [95]:
def get_ripple_amplitudes(time, filtered_lfps, ripple_times):
    ripple_stats = pd.DataFrame(columns = ['mean_amplitude', 'max_amplitude'], index = ripple_times.index) 
    
    filtered_lfps = pd.DataFrame(filtered_lfps, index=pd.Index(time, name='time'))
    for ripple in ripple_times.itertuples():
        ripple_lfps = filtered_lfps[ripple.start_time: ripple.end_time]
        ripple_lfps_amplitudes = np.max(ripple_lfps) - np.min(ripple_lfps)
        ripple_stats.loc[ripple.Index] = [np.mean(ripple_lfps_amplitudes), np.max(ripple_lfps_amplitudes)]
        
    return ripple_stats

In [96]:
def get_nadir_time(time, filtered_lfps, ripple):
    ripple_samples = np.where(np.logical_and(time >= ripple.start_time, time <= ripple.end_time))
    ripple_time = time[ripple_samples]
    ripple_lfps = filtered_lfps[ripple_samples]
    (t_idx, ch_idx) = np.unravel_index(np.argmin(ripple_lfps, axis=None), ripple_lfps.shape)
    return ripple_time[t_idx]

In [113]:
ripple_times = Eschenko_ripple_times

ripple_times['duration'] = ripple_times.apply(lambda x: x.end_time - x.start_time, axis=1)
ripple_times['center_time'] = ripple_times.apply(lambda x: x.start_time + x.duration / 2, axis=1)
ripple_times['nadir_time'] = ripple_times.apply(lambda x: get_nadir_time(time, filtered_lfps, x), axis=1)
envelope_stats = get_Kay_ripple_stats(time, filtered_lfps, sampling_frequency, ripple_times)
rms_stats = get_ripple_rms_stats(time, filtered_lfps, ripple_times)
amplitude_stats = get_ripple_amplitudes(time, filtered_lfps, ripple_times)

ripple_stats = pd.concat([ripple_times, envelope_stats, rms_stats, amplitude_stats], axis=1)
ripple_stats

,start_time,end_time,duration,center_time,nadir_time,envelope_integral,envelope_peak,mean_rms,summed_rms,max_rms,mean_amplitude,max_amplitude
ripple_number,,,,,,,,,,,,
1,9.753173e+03,9.808373e+03,55.199847,9.780773e+03,9.787573e+03,18427.7,195.497,30.4528,304.528,37.4279,137.266,162.251
2,1.037237e+04,1.043917e+04,66.799815,1.040577e+04,1.040837e+04,19173,150.141,25.2623,252.623,33.4761,113.678,155.606
3,1.046717e+04,1.049477e+04,27.599924,1.048097e+04,1.047637e+04,7678.89,148.799,25.159,251.59,29.0209,115.544,132.985
4,1.486356e+04,1.493596e+04,72.399800,1.489976e+04,1.491116e+04,25666,177.315,32.0773,320.773,35.5162,136.119,161.119
5,1.994234e+04,1.997234e+04,29.999917,1.995734e+04,1.995954e+04,9724.19,162.155,28.9759,289.759,36.4803,108.001,129.108
...,...,...,...,...,...,...,...,...,...,...,...,...
970,3.345980e+06,3.346048e+06,68.399811,3.346014e+06,3.346004e+06,20913.9,195.89,28.4821,284.821,32.6525,138.243,174.057
971,3.584062e+06,3.584114e+06,51.999856,3.584088e+06,3.584091e+06,17678.4,190.765,31.0873,310.873,44.132,148.394,191.672
972,3.586849e+06,3.586938e+06,88.799754,3.586894e+06,3.586892e+06,34539.3,255.814,35.2292,352.292,45.9811,173.695,233.177


## Export results

In [98]:
# Export the results for use in MATLAB. 
# results_file = Path('/Volumes/neuropixel/Data/CNPIX3-Valentino/2-20-2020_g0_t3.N2_ripples.csv')
results_file = Path('/Volumes/neuropixel/Data/CNPIX6-Eugene/9.24.2020_SR_24hs_g0_t0.N2_ripples.csv')
ripple_stats.to_csv(results_file)

## Explore results

In [114]:
offset_lfps = lfps - np.full(lfps.shape, np.arange(lfps.shape[1]) * 300)

In [115]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18, 6))
window_length = 1
    
@widgets.interact(ripple_number=(1, len(ripple_times), 1))
def update(ripple_number = 1):
    ax1.cla()
    ax2.cla()
    
    ripple = ripple_times.loc[ripple_number]
    window_start_time = ripple.center_time - window_length * 1000 / 2 
    window_end_time = ripple.center_time + window_length * 1000 / 2
    
    samples_to_plot = np.where(np.logical_and(time >= window_start_time, time <= window_end_time))
    ax1.plot(time[samples_to_plot], filtered_lfps[samples_to_plot], linewidth=1)
    ax2.plot(time[samples_to_plot], offset_lfps[samples_to_plot], color='black', linewidth=0.5)
    #ax2.contourf(time[samples_to_plot], np.arange(lfps.shape[1]), lfps[samples_to_plot].T)
    
    for ripple in ripple_times.itertuples():
        if (ripple.start_time >= window_start_time) and (ripple.end_time <= window_end_time):
            ax1.axvspan(ripple.start_time, ripple.end_time, alpha=0.3, color='red', zorder=1000)
            ax2.axvspan(ripple.start_time, ripple.end_time, alpha=0.3, color='red', zorder=1000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1, description='ripple_number', max=974, min=1), Output()), _dom_classes…

In [ ]:
def get_periripple_data(time, lfps, ripple_time, window_length):
    msec_per_sec = 1000
    window_start_time = ripple_time - window_length * msec_per_sec / 2 
    window_end_time = ripple_time + window_length * msec_per_sec / 2
    ripple_samples = np.where(np.logical_and(time >= window_start_time, time <= window_end_time))
    return lfps[ripple_samples], time[ripple_samples]

In [ ]:
def get_perievent_samples(event_time, time, fs, time_before, time_after):
    event_sample = (np.abs(time - event_time)).argmin()
    samples_before = np.int(time_before * fs)
    samples_after = np.int(time_after * fs)
    return (event_sample - samples_before, event_sample + samples_after)

In [ ]:
def get_perievent_time(event_time, time, fs, time_before, time_after):
    start_sample, end_sample = get_perievent_samples(event_time, time, fs, time_before, time_after)
    return time[start_sample : end_sample]

In [ ]:
def get_ripple_cwtm(ripple_lfps, fs, freq=np.linspace(1, 300, 300)):
    w = 6
    widths = w*fs / (2*freq*np.pi)

    cwtm = np.apply_along_axis(lambda sig: signal.cwt(sig, signal.morlet2, widths, w=w), 0, ripple_lfps)
    return np.mean(np.abs(cwtm), axis=2)

In [ ]:
freq = np.linspace(1, 300, 300)
time_before = 0.2
time_after = 0.2
n_expected_samples = np.int(time_before * sampling_frequency) + np.int(time_after * sampling_frequency)
ripples = [ripple for ripple in ripple_stats.itertuples() if get_perievent_time(ripple.nadir_time, time, sampling_frequency, time_before, time_after).size == n_expected_samples]
result = np.zeros((freq.size, n_expected_samples, len(ripples)))
for i, ripple in enumerate(ripples):
    start_sample, end_sample = get_perievent_samples(ripple.nadir_time, time, sampling_frequency, time_before, time_after)
    ripple_lfps = lfps[start_sample:end_sample, :]
    result[:, :, i] = get_ripple_cwtm(ripple_lfps, sampling_frequency, freq=freq)

In [ ]:
cwtm = np.mean(result, axis=2)

In [ ]:
cwtm_norm = cwtm / (1/freq)[:,None]

In [ ]:
from scipy import signal
import matplotlib.pyplot as plt

t = np.linspace(-time_before, time_after, n_expected_samples)
plt.figure()
plt.pcolormesh(t, freq, cwtm_norm, cmap='viridis', shading='gouraud')
plt.show()